# Comparison of machine learning methods for deriving $p(z,\alpha)$
This notebook establishes a framework for using machine learning methods to obtain $p(z,\alpha)$ information from training and test datasets.

In [11]:
import data
import util 
import numpy as np
from sklearn.preprocessing import RobustScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
from matplotlib import gridspec
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

We load testing and training data of the observables 'X' and unobservables 'Y'.

In [12]:
def data_vectors():

    X_test = np.loadtxt(util.dat_dir()+"X_test.dat")
    X_train = np.loadtxt(util.dat_dir()+"X_train.dat")
   
    Xerr_test = np.loadtxt(util.dat_dir()+"Xerr_test.dat")
    Xerr_train = np.loadtxt(util.dat_dir()+"Xerr_train.dat")
    
    Y_test = np.loadtxt(util.dat_dir()+"Y_test.dat")
    Y_train = np.loadtxt(util.dat_dir()+"Y_train.dat")
    
    return X_test, X_train, Y_test, Y_train

Many machine learning methods require that all variables in 'X' and 'Y' share a scaling, i.e. magnitudes must be normalized if they are in the same space as colors, because of the distance measures used in parameter space.

In [13]:
def scaler():

    X_test, X_train, Y_test, Y_train = data_vectors()
    train_X = RS.fit_transform(X_train)
    train_Y = Y_train
    test_X  = RS.transform(X_test)
    test_Y  = Y_test   
    
    return train_X, train_Y, test_X, test_Y  

We will plot the redshift, magnitude, and error distributions.

In [14]:
def plot_distribution():

    magerrs , mags, masterX, masterY  = data_vectors()

    fig = plt.figure(1, figsize=(24,8))
    gs = gridspec.GridSpec(1,3)
    ax = plt.subplot(gs[0])
    sns.distplot(masterY[:,0], kde=True, hist=False, label='Redshift')
    #ax.set_xlim([0, 2.0])
    ax.set_title('Redshift distrubtion', fontsize=18)
    ax.legend(fontsize=13)
    ax.set_xlabel('Redshift', fontsize=18)    
    ax = plt.subplot(gs[1])
    sns.distplot(masterY[:,1], kde=True, hist=False, label=r'$M_{\star}$')
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_title('stellar mass distrubtion', fontsize=18)
    ax.legend(fontsize=13)
    ax.set_xlabel('stellar mass', fontsize=18)    
    ax = plt.subplot(gs[2])
    sns.distplot(masterY[:,2], kde=True, hist=False, label=r'$SFR$')
    ax.set_xscale("log")
    ax.set_title('size', fontsize=18)
    ax.legend(fontsize=13)
    ax.set_xlabel('R', fontsize=18)    
    fig.savefig(util.fig_dir()+"z.pdf" , box_inches = "tight")
    
    fig = plt.figure(1, figsize=(8,8))
    gs = gridspec.GridSpec(1,1)
    ax = plt.subplot(gs[0])
    sns.distplot(mags[:,0], kde=True, hist=False, label = '')
    sns.distplot(mags[:,1], kde=True, hist=False, label = '')
    sns.distplot(mags[:,2], kde=True, hist=False, label = '')
    sns.distplot(mags[:,3], kde=True, hist=False, label = '')
    sns.distplot(mags[:,4], kde=True, hist=False, label = '')
    ax.set_title('magnitude distrubtion', fontsize=18)
    ax.legend(fontsize=13)
    ax.set_xlabel('magnitudes', fontsize=18)    
    fig.savefig(util.fig_dir()+"mags.pdf" , box_inches = "tight")


    fig = plt.figure(1, figsize=(8,8))
    gs = gridspec.GridSpec(1,1)
    ax = plt.subplot(gs[0])
    sns.distplot(magerrs[:,0], kde=True, hist=False, label = '')
    sns.distplot(magerrs[:,1], kde=True, hist=False, label = '')
    sns.distplot(magerrs[:,2], kde=True, hist=False, label = '')
    sns.distplot(magerrs[:,3], kde=True, hist=False, label = '')
    sns.distplot(magerrs[:,4], kde=True, hist=False, label = '')
    ax.set_title('magerrs', fontsize=18)
    ax.legend(fontsize=13)
    ax.set_xlabel('magerrs', fontsize=18)    
    fig.savefig(util.fig_dir()+"magerrs.pdf" , box_inches = "tight")

    return None

plot_distribution()

We choose a top hat function parametrization of $z,\alpha$ space for the probability distributions.  At this point we hardcode that $\alpha$ is a scalar that we'll refer to as $M$ in this notebook because our first stab was with $\alpha=M_{*}$.

In [15]:
def make_bins(Y_train,num_bins):
    
    (nbins_z,nbins_M) = num_bins
    
    zmin, zmax = min(Y_train[:,0]), max(Y_train[:,0])
    Mmin, Mmax = min(Y_train[:,1]), max(Y_train[:,1])
   
    zgrid = np.linspace(zmin, zmax, nbins_z+1)
    Mgrid = np.linspace(Mmin, Mmax, nbins_M+1)
    
    X_train, Y_train, test_X, test_Y = scaler()
    
    (nbins_z,nbins_M) = num_bins
    (bins_z,bins_M) = zgrid,Mgrid#make_bins(Y_train,num_bins)
    zmin,zmax = min(bins_z),max(bins_z)
    Mmin,Mmax = min(bins_M),max(bins_M)
    binsize_z = (zmax - zmin)/ nbins_z
    binsize_M = (Mmax - Mmin)/ nbins_M

    L = np.zeros((len(Y_train)))
    label_list = [] 
    """ Labeling the bins """
    for i in range(nbins_z):
        for j in range(nbins_M):
            ij = np.where((Y_train[:,0]>=zmin+i*binsize_z)&(Y_train[:,0]<zmin+(i+1)*binsize_z)&(Y_train[:,1]>=Mmin+j*binsize_M)&(Y_train[:,1]<Mmin+(j+1)*binsize_M))[0]
            if (len(ij) != 0 ):
                label_list.append([i,j])
            L[ij] = i + j * nbins_M
    
    return ((zmin , binsize_z, Mmin, binsize_M),(L,label_list))#(zgrid,Mgrid)

This is the class that implements two machine learning methods but could easily be extended to any others.

In [16]:
class classifier(object):
    def __init__(self,name,num_bins,**kwargs):
        
        """ Setting up the KNN classifier"""
        ((self.zmin , self.binsize_z, self.Mmin, self.binsize_M),(self.L,self.label_list)) = make_bins(Y_train,num_bins)
        
        if name == 'knn':
            """
            k nearest neighbors classifier for determining p(z)'s.
            inputs:
            num_bins = number of redshift bins,
            num_neighbors = number of neighbors,
            outputs:
            label probabilities + best predictions + feature importance
            """
            self.num_neighbors = kwargs['num_neighbors']
            self.clf = KNeighborsClassifier(n_neighbors=self.num_neighbors)
        if name == 'rf':
            """
            random forest classifier for determining p(z)'s.
            inputs:
            num_bins = number of redshift bins,
            num_estimators = number of estimators,
            outputs:
            label probabilities + best predictions + feature importance
            """
            self.num_estimators = kwargs['num_estimators']
            self.clf = RandomForestClassifier(n_estimators=self.num_estimators, 
                                              max_depth=None, min_samples_split=1, random_state=0)
        return
            
    def run(self):
        self.clf.n_classes_ = len(self.label_list)
        """ training """
        self.clf.fit(X_train, self.L)
        """best predictions"""
        Y_pred = self.clf.predict(test_X)
        """label probabilities"""
        prob = self.clf.predict_proba(test_X) 
        """transformation quantities """
        trans = self.zmin , self.binsize_z, self.Mmin, self.binsize_M
        return self.L, self.label_list, prob , Y_pred, trans

Now we run the classifiers as pseudo-regressors.  Random forests is highly memory intensive and sensitive to the number of classes.  $k$ nearest neighbors is faster and much more forgiving with the granularity of the space, but it is sensitive to the sparsity of the cells in training set space.

In [17]:
from matplotlib.colors import LogNorm
  
RS = RobustScaler()
    
X_train, Y_train, test_X, test_Y = scaler()

nbins_knn = (10,25)
nbins_rf = (10,10)
nn = 100
ne = 100
    
clf_knn = classifier('knn', nbins_knn, num_neighbors = nn)
clf_rf = classifier('rf', nbins_rf, num_estimators = ne)

L_knn, label_list_knn, prob_knn, bestfit_knn, trans_knn = clf_knn.run() 
L_rf, label_list_rf, prob_rf, bestfit_rf, trans_rf = clf_rf.run()

zmin_knn, binsize_z_knn, Mmin_knn, binsize_M_knn = trans_knn
zmin_rf, binsize_z_rf, Mmin_rf, binsize_M_rf = trans_rf

zrange_knn = zmin_knn + binsize_z_knn * np.arange(nbins_knn[0])
mrange_knn = Mmin_knn + binsize_M_knn * np.arange(nbins_knn[1])
zrange_rf = zmin_rf + binsize_z_rf * np.arange(nbins_rf[0])
mrange_rf = Mmin_rf + binsize_M_rf * np.arange(nbins_rf[1])

#print(zrange.shape, mrange.shape, prob.shape, bestfit)

For every galaxy in the catalog, we plot the posteriors $p(z,\alpha)$ and the marginal posteriors $p(z)$ and $p(\alpha)$ with the true values.  Some will show nontrivial covariance, which is the motivation for why $p(z,\alpha)$ is an informative data product.

In [18]:
def plot_results(name,nbins,zrange,mrange,L,label_list,prob):
    print(np.shape(L))
    print(np.shape(label_list))
    print(np.shape(prob))
    for i in range(prob[:10].shape[0]):
    
       prob2d = np.zeros(nbins)
    
       for k in xrange(len(prob[i])):
           
           prob2d[label_list[k][0] , label_list[k][1]] = prob[i][k]

       image = prob2d#[0][i,:][:,None] * prob[1][i,:][None,:] 
       image = image / np.sum(image)     
       #plt.title(str())
       plt.imshow(image , interpolation = "none", cmap = plt.cm.viridis, origin='lower')#,norm=LogNorm(vmin=-0.0000001, vmax=1))
       ytick_locs = range(nbins[0])
       ytick_lbls = np.round(zrange,2)
       plt.yticks(ytick_locs, ytick_lbls)
       plt.ylabel(r'$z$')
       xtick_locs = range(nbins[1])
       xtick_lbls = np.round(mrange,2)
       plt.xticks(xtick_locs, xtick_lbls, rotation='vertical')
       plt.xlabel(r'$M_{*}$')
       plt.colorbar()
       plt.savefig(util.fig_dir()+name+str(i)+".png")
       plt.close()
    
       plt.plot(mrange, np.array(np.sum(prob2d,axis=0)), color='blue' , drawstyle='steps-mid')
       plt.axvline(x=test_Y[i,1], color='k', linestyle='--') 
       plt.xlabel(r'$M_{*}$')
       plt.savefig(util.fig_dir()+name+str(i)+"m.png")
       plt.close()
       plt.plot(zrange, np.array(np.sum(prob2d,axis=1)), color='blue' , drawstyle='steps-mid')
       plt.axvline(x=test_Y[i,0], color='k', linestyle='--') 
       plt.xlabel(r'$z$')
       plt.savefig(util.fig_dir()+name+str(i)+"z.png")
       plt.close()

In [19]:
plot_results('knn',nbins_knn,zrange_knn,mrange_knn,L_knn,label_list_knn,prob_knn)
plot_results('rf',nbins_rf,zrange_rf,mrange_rf,L_rf,label_list_rf,prob_rf)

(37009,)
(117, 2)
(44886, 117)
(37009,)
(56, 2)
(44886, 56)
